In [2]:
import datetime
import httplib2
import os
import re
import time

from apiclient import discovery
from oauth2client import file, client, tools
from tqdm import tqdm

In [3]:
NEW_ORDERS_LABEL = 'Label_7015697172796597976'

In [4]:
def get_credentials(client_secret_file, scopes):
    cwd_dir = os.getcwd()
    credential_dir = os.path.join(cwd_dir, 'infrastructure')
    if not os.path.exists(credential_dir):
        os.makedirs(credential_dir)

    credential_path = os.path.join(credential_dir, 'gmail-credentials.json')

    store = file.Storage(credential_path)
    credentials = store.get()

    if not credentials or credentials.invalid:
        flow = client.flow_from_clientsecrets(client_secret_file, scopes)
        credentials = tools.run_flow(flow, store)
        print('Storing credentials to ' + credential_path)

    return credentials

In [5]:
def build_service(credentials):
    http = httplib2.Http()
    http = credentials.authorize(http)

    return discovery.build('gmail', 'v1', http=http)

In [6]:
def connect_gmail():
    scopes = ['https://www.googleapis.com/auth/gmail.readonly',
              'https://www.googleapis.com/auth/gmail.modify',
              'https://www.googleapis.com/auth/gmail.labels']
    cwd_dir = os.getcwd()
    client_secret_dir = os.path.join(cwd_dir, 'infrastructure')
    client_secret_file = os.path.join(client_secret_dir, 'oauth_client_id.json')

    credentials = get_credentials(client_secret_file, scopes)
    service = build_service(credentials)

    return service

In [7]:
def list_threads(service, label_ids):
    response = service.users().threads().list(userId='me', labelIds=label_ids).execute()

    threads = []
    if 'threads' in response:
        threads.extend(response['threads'])

    while 'nextPageToken' in response:
        page_token = response['nextPageToken']
        response = service.users().threads().list(userId="me", labelIds=label_ids, pageToken=page_token).execute()
        threads.extend(response['threads'])

    return threads

In [8]:
def print_labels(service):
    response = service.users().labels().list(userId='me').execute()
    labels = response['labels']

    if not labels:
        print('No labels found.')
    else:
        print('Labels:')
        for label in labels:
            print(label['name'] + "," + label['id'])

In [9]:
start = time.time()
print(f"Execution start: {datetime.datetime.now()}")
service = connect_gmail()
# print_labels(service)
threads = list_threads(service, [NEW_ORDERS_LABEL])
# print(f"Threads: {threads}")
for thread in tqdm(threads):
    # print(thread['id'])
    thread = service.users().threads().get(userId="me", id=thread['id']).execute()
    for message in thread['messages']:
        print(message['id'])
        headers = message['payload']['headers']
        
        correo = {}

        for header in headers:
            if header['name'] == 'Subject':
                print(f"Subject: {header['value']}")
                correo['Subject'] = header['value']
            elif header['name'] == 'Date':
                print(f"Date: {header['value']}")
                correo['Date'] = datetime.datetime.strptime(header['value'], "%a, %d %b %Y %H:%M:%S %z")
            elif header['name'] == 'From':
                print(f"From: {re.findall(r'<(.*)>', header['value'])[0]}")
                correo['From'] = re.findall(r'<(.*)>', header['value'])[0]

Execution start: 2018-10-26 23:31:17.684045


  0%|          | 0/419 [00:00<?, ?it/s]

  0%|          | 1/419 [00:00<01:21,  5.13it/s]

  0%|          | 2/419 [00:00<01:19,  5.26it/s]

166b0c6e18437072
From: akayaba@aitworldwide.com
Subject: Pick up request//S00305200//235-32500834
Date: Fri, 26 Oct 2018 14:28:16 +0000
166b0ca2ac283187
From: dispatch@hawkxinc.com
Date: Fri, 26 Oct 2018 10:31:52 -0400
Subject: Re: Pick up request//S00305200//235-32500834
166ac4c4a3e18cf5
From: jogarcia@aitworldwide.com
Subject: FW: AIT WORLDWIDE LOGISTICS, INC. - Pickup Order - S00305417
Date: Thu, 25 Oct 2018 17:35:58 +0000
166ac4fd8477de57
From: logistic@hawkxinc.com
Date: Thu, 25 Oct 2018 13:39:34 -0400
Subject: Re: FW: AIT WORLDWIDE LOGISTICS, INC. - Pickup Order - S00305417


...

1668e42126e1ba5e
From: jogarcia@aitworldwide.com
Subject: FW: AIT WORLDWIDE LOGISTICS, INC. - Pickup Order - S00304177
Date: Fri, 19 Oct 2018 21:36:15 +0000
1668e44522b2cb53
From: dispatch@hawkxinc.com
Date: Fri, 19 Oct 2018 17:38:38 -0400
Subject: Re: FW: AIT WORLDWIDE LOGISTICS, INC. - Pickup Order - S00304177
1668e45e6f3a5f6d
From: jogarcia@aitworldwide.com
Subject: RE: FW: AIT WORLDWIDE LOGISTICS, INC. - Pickup Order - S00304177
Date: Fri, 19 Oct 2018 21:40:28 +0000
1668e471af872a4a
From: dispatch@hawkxinc.com
Date: Fri, 19 Oct 2018 17:41:41 -0400
Subject: Re: FW: AIT WORLDWIDE LOGISTICS, INC. - Pickup Order - S00304177
1668e7c73cf13ce3
From: jogarcia@aitworldwide.com
Subject: RE: FW: AIT WORLDWIDE LOGISTICS, INC. - Pickup Order - S00304177
Date: Fri, 19 Oct 2018 22:39:57 +0000
1668e7f9cf2145e1
From: logistic@hawkxinc.com
Date: Fri, 19 Oct 2018 18:43:26 -0400
Subject: Re: FW: AIT WORLDWIDE LOGISTICS, INC. - Pickup Order - S00304177
1668e83aca9d6315
From: jogarcia@aitworldwide.com
S

  3%|▎         | 11/419 [00:02<01:20,  5.09it/s]

166872ffde8c27c6
From: clperez@aitworldwide.com
Subject: Recall: S00296825/ AIRLINE DROP CU OFF 10 AM
Date: Thu, 18 Oct 2018 12:39:15 +0000


  3%|▎         | 12/419 [00:02<01:58,  3.43it/s]

16669d0e7c1ded90
From: clperez@aitworldwide.com
Subject: SATURDAY DROP/ LAN/ S00296825
Date: Fri, 12 Oct 2018 19:46:20 +0000
16669dcaeb9ea531
From: dispatch@hawkxinc.com
Date: Fri, 12 Oct 2018 15:59:06 -0400
Subject: Re: SATURDAY DROP/ LAN/ S00296825
16669defdbedffae
From: clperez@aitworldwide.com
Subject: RE: SATURDAY DROP/ LAN/ S00296825
Date: Fri, 12 Oct 2018 20:01:46 +0000
16669eb99842fb96
From: dispatch@hawkxinc.com
Date: Fri, 12 Oct 2018 16:15:25 -0400
Subject: Re: SATURDAY DROP/ LAN/ S00296825
1666a0164cc4b1a3
From: clperez@aitworldwide.com
Subject: RE: SATURDAY DROP/ LAN/ S00296825
Date: Fri, 12 Oct 2018 20:39:20 +0000
1666a0c96dfa39f7
From: dispatch@hawkxinc.com
Date: Fri, 12 Oct 2018 16:51:25 -0400
Subject: Re: SATURDAY DROP/ LAN/ S00296825
1666dfcbbfcbbc8e
From: dispatch@hawkxinc.com
Date: Sat, 13 Oct 2018 11:12:36 -0400
Subject: Re: SATURDAY DROP/ LAN/ S00296825
1666e1339438bab3
From: dispatch@hawkxinc.com
Date: Sat, 13 Oct 2018 11:37:08 -0400
Subject: Re: SATURDAY DROP/ LA